In [ ]:
# !pip install -q -U google-generativeai
# !pip install -q pandas

In [56]:
import json
import pandas as pd
import os
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

## Method 2

1.   pass in image table into VLM
2.   get json output of basic table structure as is
3.   search through json output OR expand all combinations programmatically

### prompts

In [55]:
# system_instruction = "I will ask you to parse tables in the form of images that contain pricing information for different types of furniture, for eg tables (tavolos). Each table contains several prices based on attributes for that product, such as color, finishing, material, dimensions. These tables are built by the brand behind the product and thus, tables will look different based on their choices. For example, some might prefer to compress data for different attributes into one cell if the price for those attributes is the same. Others might not use clear boundary lines in the table. Your task will be to accurately extract all combinations of attributes from the table & accurately identify the price for that combination. Then you will output a list of JSON objects, each object representing one combination containing attributes and their values."
# system_instruction = "you take image inputs that contain tables and just output the full data in json format as instructed"
system_instruction = "You are a table data extraction system. Process the ENTIRE table and output ALL combinations in JSON format. Do not truncate, summarize, or ask for confirmation. Output the complete data in one response."

prompt_text_old = """
I want you to parse a furniture pricing table from the image attached and output the data in JSON format.
The table describes combinations of attributes, each represented by row and column headers. Your task is to produce the output below

TASK INPUT:
An image of a complex table with pricing information based on several attributes, such as color codes, finishing, dimensions, etc.
There might be nested sub-categories and some columns might be combined

TASK OUTPUT:
A JSON array of objects, where each object represents the price for each combination of attributes in the table. The JSON format should be:
[
  {
    attribute 1: attribute 1 value,
    attribute 2: attribute 2 value,
    attribute 3: attribute 3 value,
    attribute X: attribute X value,
    price: price in EUR from image
  },
  ... // More objects for other combinations
]
"""

prompt_text = """
I want you to parse a furniture pricing table from the image attached and output the data in JSON format.
The table describes combinations of attributes, each represented by row and column headers. Your task is to produce the output below

TASK INPUT: 
An image of a complex table with pricing information based on several attributes, such as color codes, finishing, dimensions, etc.
There might be nested sub-categories and some columns might be combined

TASK OUTPUT: 
A JSON array of objects, where each object represents the price for each combination of attributes in the table. Output the complete list of objects. The JSON format should be:
[
  {
    attribute 1: attribute 1 value,
    attribute 2: attribute 2 value,
    attribute 3: attribute 3 value,
    attribute X: attribute X value,
    price: price in EUR from image
  }
]

<examples>
<example>
input:
"""
prompt_text2 = """
output:
[{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Ecopelle / C.O.M.","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"705"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Tessuto / Synthetic nubuck / Micro nubuck / C.O.L","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"738"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"818"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle Glove","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"858"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Ecopelle / C.O.M.","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"818"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Tessuto / Synthetic nubuck / Micro nubuck / C.O.L","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"871"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"972"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle Glove","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"1.023"}]
</example>

<example>
input:
"""

prompt_text3 = """
output:
[{"Top":"NC / RB","Base":"GFM69 / GFM73 - 06","Dimensions_CM":"B 250x128x75h","Dimensions_INCHES":"98³/₈x50³/₈x29¹/₂h","M3":"1,05","Colli":"3","EUR":"3.570"},{"Top":"NC / RB","Base":"GFM11 / GFM18 - 06","Dimensions_CM":"B 250x128x75h","Dimensions_INCHES":"98³/₈x50³/₈x29¹/₂h","M3":"1,05","Colli":"3","EUR":"3.589"},{"Top":"NC / RB","Base":"GFM69 / GFM73 - 06","Dimensions_CM":"B 300x128x75h","Dimensions_INCHES":"118¹/₈x50³/₈x29¹/₂h","M3":"1,13","Colli":"4","EUR":"3.719"},{"Top":"NC / RB","Base":"GFM11 / GFM18 - 06","Dimensions_CM":"B 300x128x75h","Dimensions_INCHES":"118¹/₈x50³/₈x29¹/₂h","M3":"1,13","Colli":"4","EUR":"3.738"}]
</example>

<example>
input:
"""

prompt_text4 = """
output:
[{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"S1 ø62x66h","Dimensions_INCHES":"ø24³/₈x26h","M3":"0,33","Colli":"1","EUR":"764"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"S1 ø62x66h","Dimensions_INCHES":"ø24³/₈x26h","M3":"0,33","Colli":"1","EUR":"787"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"S2 ø82x73h","Dimensions_INCHES":"ø32¹/₄x28³/₄h","M3":"0,67","Colli":"1","EUR":"1.313"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"S2 ø82x73h","Dimensions_INCHES":"ø32¹/₄x28³/₄h","M3":"0,67","Colli":"1","EUR":"1.351"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"P1 ø62x138h","Dimensions_INCHES":"ø24³/₈x54³/₈h","M3":"0,74","Colli":"1","EUR":"1.082"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"P1 ø62x138h","Dimensions_INCHES":"ø24³/₈x54³/₈h","M3":"0,74","Colli":"1","EUR":"1.112"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"P2 ø82x149h","Dimensions_INCHES":"ø32¹/₄x58⁵/₈h","M3":"1,34","Colli":"1","EUR":"1.455"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"P2 ø82x149h","Dimensions_INCHES":"ø32¹/₄x58⁵/₈h","M3":"1,34","Colli":"1","EUR":"1.500"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"Cavo aggiuntivo 1 mt versione S","EUR":"17"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"Cavo aggiuntivo 1 mt versione S","EUR":"39"}]
</example>
</examples>
"""

few_shot_examples = [
    {
        "input_text": "EXAMPLE 2:",
        "image_path": "./few-shot-examples/table-image-chair.png",
        "output": """
          [{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Ecopelle / C.O.M.","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"705"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Tessuto / Synthetic nubuck / Micro nubuck / C.O.L","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"738"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"818"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle Glove","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"858"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Ecopelle / C.O.M.","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"818"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Tessuto / Synthetic nubuck / Micro nubuck / C.O.L","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"871"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"972"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle Glove","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"1.023"}]
        """
    },
    {
        "input_text": "EXAMPLE 1:",
        "image_path": "./few-shot-examples/table-image-desk2.png",
        "output": """
          [{"Top":"NC / RB","Base":"GFM69 / GFM73 - 06","Dimensions_CM":"B 250x128x75h","Dimensions_INCHES":"98³/₈x50³/₈x29¹/₂h","M3":"1,05","Colli":"3","EUR":"3.570"},{"Top":"NC / RB","Base":"GFM11 / GFM18 - 06","Dimensions_CM":"B 250x128x75h","Dimensions_INCHES":"98³/₈x50³/₈x29¹/₂h","M3":"1,05","Colli":"3","EUR":"3.589"},{"Top":"NC / RB","Base":"GFM69 / GFM73 - 06","Dimensions_CM":"B 300x128x75h","Dimensions_INCHES":"118¹/₈x50³/₈x29¹/₂h","M3":"1,13","Colli":"4","EUR":"3.719"},{"Top":"NC / RB","Base":"GFM11 / GFM18 - 06","Dimensions_CM":"B 300x128x75h","Dimensions_INCHES":"118¹/₈x50³/₈x29¹/₂h","M3":"1,13","Colli":"4","EUR":"3.738"}]
        """
    },
    {
        "input_text": "EXAMPLE 3:",
        "image_path": "./few-shot-examples/table-image-lamp.png",
        "output": """
          [{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"S1 ø62x66h","Dimensions_INCHES":"ø24³/₈x26h","M3":"0,33","Colli":"1","EUR":"764"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"S1 ø62x66h","Dimensions_INCHES":"ø24³/₈x26h","M3":"0,33","Colli":"1","EUR":"787"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"S2 ø82x73h","Dimensions_INCHES":"ø32¹/₄x28³/₄h","M3":"0,67","Colli":"1","EUR":"1.313"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"S2 ø82x73h","Dimensions_INCHES":"ø32¹/₄x28³/₄h","M3":"0,67","Colli":"1","EUR":"1.351"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"P1 ø62x138h","Dimensions_INCHES":"ø24³/₈x54³/₈h","M3":"0,74","Colli":"1","EUR":"1.082"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"P1 ø62x138h","Dimensions_INCHES":"ø24³/₈x54³/₈h","M3":"0,74","Colli":"1","EUR":"1.112"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"P2 ø82x149h","Dimensions_INCHES":"ø32¹/₄x58⁵/₈h","M3":"1,34","Colli":"1","EUR":"1.455"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"P2 ø82x149h","Dimensions_INCHES":"ø32¹/₄x58⁵/₈h","M3":"1,34","Colli":"1","EUR":"1.500"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"Cavo aggiuntivo 1 mt versione S","EUR":"17"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"Cavo aggiuntivo 1 mt versione S","EUR":"39"}]
        """
    }
]

print(few_shot_examples[0]["image_path"])
print(few_shot_examples[1]["image_path"])
print(few_shot_examples[2]["image_path"])

few_shot_examples_gpt = [
    {
        "input_text": f"EXAMPLE 1:{prompt_text}",
        "image_path": "https://smartcatalog.blob.core.windows.net/table-images/table-image-desk2.png",
        "output": """
          [{"Top":"NC / RB","Base":"GFM69 / GFM73 - 06","Dimensions_CM":"B 250x128x75h","Dimensions_INCHES":"98³/₈x50³/₈x29¹/₂h","M3":"1,05","Colli":"3","EUR":"3.570"},{"Top":"NC / RB","Base":"GFM11 / GFM18 - 06","Dimensions_CM":"B 250x128x75h","Dimensions_INCHES":"98³/₈x50³/₈x29¹/₂h","M3":"1,05","Colli":"3","EUR":"3.589"},{"Top":"NC / RB","Base":"GFM69 / GFM73 - 06","Dimensions_CM":"B 300x128x75h","Dimensions_INCHES":"118¹/₈x50³/₈x29¹/₂h","M3":"1,13","Colli":"4","EUR":"3.719"},{"Top":"NC / RB","Base":"GFM11 / GFM18 - 06","Dimensions_CM":"B 300x128x75h","Dimensions_INCHES":"118¹/₈x50³/₈x29¹/₂h","M3":"1,13","Colli":"4","EUR":"3.738"}]
        """
    },
    {
        "input_text": f"EXAMPLE 2:{prompt_text}",
        "image_path": "https://smartcatalog.blob.core.windows.net/table-images/table-image-chair.png",
        "output": """
          [{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Ecopelle / C.O.M.","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"705"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Tessuto / Synthetic nubuck / Micro nubuck / C.O.L","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"738"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"818"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle Glove","Dimensions_CM":"53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,48","Colli":"2","EUR":"858"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Ecopelle / C.O.M.","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"818"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Tessuto / Synthetic nubuck / Micro nubuck / C.O.L","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"871"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"972"},{"Telaio":"frN / frNC / frRB / fr71 / fr73","Seduta":"Pelle Glove","Dimensions_CM":"B 53,5x59x86h","Dimensions_INCHES":"21¹/₈x23¹/₄x33⁷/₈h","M3":"0,36","Colli":"1","EUR":"1.023"}]
        """
    },
    {
        "input_text": f"EXAMPLE 3:{prompt_text}",
        "image_path": "https://smartcatalog.blob.core.windows.net/table-images/table-image-lamp.png",
        "output": """
          [{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"S1 ø62x66h","Dimensions_INCHES":"ø24³/₈x26h","M3":"0,33","Colli":"1","EUR":"764"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"S1 ø62x66h","Dimensions_INCHES":"ø24³/₈x26h","M3":"0,33","Colli":"1","EUR":"787"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"S2 ø82x73h","Dimensions_INCHES":"ø32¹/₄x28³/₄h","M3":"0,67","Colli":"1","EUR":"1.313"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"S2 ø82x73h","Dimensions_INCHES":"ø32¹/₄x28³/₄h","M3":"0,67","Colli":"1","EUR":"1.351"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"P1 ø62x138h","Dimensions_INCHES":"ø24³/₈x54³/₈h","M3":"0,74","Colli":"1","EUR":"1.082"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"P1 ø62x138h","Dimensions_INCHES":"ø24³/₈x54³/₈h","M3":"0,74","Colli":"1","EUR":"1.112"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"P2 ø82x149h","Dimensions_INCHES":"ø32¹/₄x58⁵/₈h","M3":"1,34","Colli":"1","EUR":"1.455"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"P2 ø82x149h","Dimensions_INCHES":"ø32¹/₄x58⁵/₈h","M3":"1,34","Colli":"1","EUR":"1.500"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"220V","Dimensions_CM":"Cavo aggiuntivo 1 mt versione S","EUR":"17"},{"Struttura":"OP17 / Nichel","Rivestimento":"Avorio / Marrone / Nero","Impianto":"110V","Dimensions_CM":"Cavo aggiuntivo 1 mt versione S","EUR":"39"}]
        """
    }
]

./few-shot-examples/table-image-chair.png
./few-shot-examples/table-image-desk2.png
./few-shot-examples/table-image-lamp.png


## Claude prompt setup, processing, calling

import anthropic and API keys

In [15]:
from anthropic import Anthropic
import base64

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
client = Anthropic(api_key=anthropic_api_key)


In [16]:
# image encoding
def encode_image_to_base64(image_path):
    with open(image_path, 'rb') as image_file:
        return base64.standard_b64encode(image_file.read()).decode("utf-8")

create prompt function

In [17]:
def build_prompt(prompt_text, prompt_image, few_shot_examples=None):
    """Build the complete messages array"""

    # # add pre-fill response so it can get to the fkn point
    # messages.extend([{"role": "assistant", "content": "["}])

    # LEFT HERE
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": encode_image_to_base64(prompt_image)
                    },
                },
                {
                    "type": "text",
                    "text": prompt_text
                },
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": encode_image_to_base64(few_shot_examples[0]["image_path"])
                    },
                },
                {
                    "type": "text",
                    "text": prompt_text2
                },
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": encode_image_to_base64(few_shot_examples[1]["image_path"])
                    },
                },
                {
                    "type": "text",
                    "text": prompt_text3
                },
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/png",
                        "data": encode_image_to_base64(few_shot_examples[2]["image_path"])
                    },
                },
                {
                    "type": "text",
                    "text": prompt_text4
                }
            ],
        },
        {"role": "assistant", "content": "["} # pre-fill response so it can get to the fkn point
    ]
    return messages


In [52]:
def process_image_claude(prompt_image, prompt_text, sys_instruction=""):
    """Main function to process image with Claude"""
    try:
        messages = build_prompt(prompt_text, prompt_image, few_shot_examples)
        response = client.messages.create(
            model="claude-3-5-sonnet-20241022",
            system=sys_instruction,
            messages=messages,
            temperature=0.5,
            max_tokens=8192
        )
        result = response.content[0].text.strip()
        if result.startswith("{"):
            return "[" + result
        else:
            return result
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        return None

call claude

In [ ]:
image_url = "./table-image-desk3.png"  # Replace with your image URL
response = process_image_claude(image_url, prompt_text, system_instruction)
print(response)

In [54]:
# TESTING OUTPUTS
full_response = ""
if response.strip().startswith("{"):
    full_response = "[" + response
else:
    full_response = response

# print(full)
data_json = json.loads(full_response)
print(len(data_json))
pd.DataFrame.from_records(data_json)


28


,Base,Top,Dimensions_CM,Dimensions_INCHES,M3,Colli,EUR
0,GFM69 / GFM73,KM02 / KM04 / KM05 / KM06 / KM09 / KM10 / KM11...,"200x106x73,5h",78³/₄x41³/₄x28⁷/₈h,"0,79",5,3.458
1,GFM69 / GFM73,KM07 / KM08,"200x100x73,5h",78³/₄x39³/₈x28⁷/₈h,"0,77",5,3.437
2,GFM69 / GFM73,KM02 / KM04 / KM05 / KM06 / KM09 / KM10 / KM11...,"200x120x73,5h sag",78³/₄x47¹/₄x28⁷/₈h shaped,"0,86",5,3.711
3,GFM69 / GFM73,KM07 / KM08,"200x120x73,5h sag",78³/₄x47¹/₄x28⁷/₈h shaped,"0,86",5,3.711
4,GFM69 / GFM73,KM02 / KM04 / KM05 / KM06 / KM09 / KM10 / KM11...,"240x120x73,5h",94¹/₂x47¹/₄x28⁷/₈h,"0,90",5,3.717
5,GFM69 / GFM73,KM07 / KM08,"240x120x73,5h",94¹/₂x47¹/₄x28⁷/₈h,"0,90",5,3.717
6,GFM69 / GFM73,KM02 / KM04 / KM05 / KM06 / KM09 / KM10 / KM11...,"240x120x73,5h sag",94¹/₂x47¹/₄x28⁷/₈h shaped,"0,90",5,3.852
7,GFM69 / GFM73,KM07 / KM08,"240x120x73,5h sag",94¹/₂x47¹/₄x28⁷/₈h shaped,"0,90",5,3.852
8,GFM69 / GFM73,KM02 / KM04 / KM05 / KM06 / KM09 / KM10 / KM11...,"300x120x73,5h",118¹/₈x47¹/₄x28⁷/₈h,"1,06",5,4.873
9,GFM69 / GFM73,KM07 / KM08,"300x120x73,5h",118¹/₈x47¹/₄x28⁷/₈h,"1,06",5,4.873


### gemini setup, file upload, processing

In [ ]:
import os
import google.generativeai as genai

# Create the model
generation_config = {
  "response_mime_type": "application/json",
  "temperature": 0.8
}

genai.configure(api_key=userdata.get('gemini_api_key'))
model = genai.GenerativeModel(
    "gemini-1.5-pro-002"
    # generation_config=generation_config
    # system_instruction=""
)

# UPLOAD TABLE IMAGE TO GEMINI
def upload_to_gemini(path, mime_type=None):
  # """Uploads the given file to Gemini.
  # See https://ai.google.dev/gemini-api/docs/prompting_with_media
  # """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def process_image_gemini(image_path, prompt, examples=None, generation_config=None):
    # Upload the image to Gemini
    target_image = upload_to_gemini(image_path, mime_type="image/png")

    prompt_parts = []
    # add prompt
    prompt_parts.extend([
        target_image,
        prompt
    ])
    # add examples
    if examples:
        for example in examples:
            example_image = upload_to_gemini(example['image_path'], mime_type="image/png")

            # Add example input (image and text)
            prompt_parts.extend([
                "<EXAMPLE>\nINPUT:",
                example['input_text'],
                example_image,
                "\nOUTPUT:",
                example['output'],
                "\n</EXAMPLE>\n---\n"
            ])

    # print(prompt_parts)

    # Create the prompt object
    prompt_obj = {
        "parts": prompt_parts
    }

    # Generate content using the Gemini model
    if generation_config:
        response = model.generate_content(prompt_obj, generation_config=generation_config)
    else:
        response = model.generate_content(prompt_obj)

    print(response.text)
    return response.text


In [ ]:
image1 = "/content/table-image.png"
image_data = process_image_gemini(image1, prompt_text, examples=few_shot_examples, generation_config=generation_config)
image2 = "/content/table-image-desk3.png"
image_data_desk3 = process_image_gemini(image2, prompt_text, examples=few_shot_examples, generation_config=generation_config)
image3 = "/content/table-image-madia.png"
image_data_madia = process_image_gemini(image3, prompt_text, examples=few_shot_examples, generation_config=generation_config)

Uploaded file 'table-image.png' as: https://generativelanguage.googleapis.com/v1beta/files/t27edmrjeohs
Uploaded file 'table-image-desk2.png' as: https://generativelanguage.googleapis.com/v1beta/files/izcy8w5f5c2x
Uploaded file 'table-image-chair.png' as: https://generativelanguage.googleapis.com/v1beta/files/7iszhlgeconv
Uploaded file 'table-image-lamp.png' as: https://generativelanguage.googleapis.com/v1beta/files/83q7clnj2f98
[{"Top":"CrystalMatt CY01 / CY02","Base":"GFM69 / GFM73 - 06","MISURA_CM":"240x120x74h","INCHES":"94¹/₂x47¹/₄x29¹/₈h","m³":"0,98","colli":"3","EUR":"3.192"},{"Top":"CrystalMatt CY01 / CY02","Base":"GFM11 / GFM18 - 06","MISURA_CM":"240x120x74h","INCHES":"94¹/₂x47¹/₄x29¹/₈h","m³":"0,98","colli":"3","EUR":"3.211"},{"Top":"CrystalMatt CY01 / CY02","Base":"GFM69 / GFM73 - 06","MISURA_CM":"240x120x74h sag","INCHES":"94¹/₂x47¹/₄x29¹/₈h shaped","m³":"0,98","colli":"3","EUR":"3.370"},{"Top":"CrystalMatt CY01 / CY02","Base":"GFM11 / GFM18 - 06","MISURA_CM":"240x120x74h s

In [ ]:
import json
import pandas as pd

data_json = json.loads(image_data)
print(data_json)
print(len(data_json))

data_json2 = json.loads(image_data_desk3)
print(len(data_json2))

data_json3 = json.loads(image_data_madia)
print(len(data_json3))

NameError: name 'image_data' is not defined

In [ ]:
pd.DataFrame.from_records(data_json)

ValueError: If using all scalar values, you must pass an index

### GPT-4o prompt setup, processing function, calling

In [ ]:
def create_prompt_gpt4o(prompt_image, prompt_text, few_shot_examples=None, sys_instruction=None):
  prompt = []
  # Add the system instruction
  if sys_instruction:
    prompt.append({"role": "system", "content": sys_instruction})

  # add the few shot examples
  for example in few_shot_examples:
    prompt.append({"role": "user", "content": [{"type": "text", "text": example["input_text"]}, {"type": "image_url", "image_url": { "url": example["image_path"] }}]})
    prompt.append({"role": "assistant", "content": example["output"]})

  # Add the actual user prompt
  prompt.append({"role": "user", "content": [
      {
          "type": "text",
          "text": prompt_text
      },
      {
          "type": "image_url",
          "image_url": { "url": prompt_image }
      }
  ]})
  return prompt

In [ ]:
import openai
from openai import AzureOpenAI
from google.colab import userdata

OAI_API_KEY = userdata.get('gpt_api_key')
# endpoint = "https://gpt-document-parsing.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview"
ENDPOINT = "https://gpt-document-parsing.openai.azure.com/"
deployment_model = "gpt-4o"

# Set up your Azure OpenAI client
client = AzureOpenAI(
    api_key=OAI_API_KEY,  # Replace with your Azure OpenAI API key
    azure_endpoint=ENDPOINT,  # Replace with your Azure endpoint
    api_version="2024-08-01-preview"  # Ensure the API version matches the one you set up in Azure
)

# Gpt 4o API request function
def process_image_gpt4o(image_path, prompt_input, examples=None):
    # you put together the prompt including the few-shot examples with images
    prompt = create_prompt_gpt4o(image_path, prompt_input, examples)
    print(prompt)

    try:
        # Send the request to the OpenAI API
        response = client.chat.completions.create(
            model=deployment_model,  # Replace with your deployment model name
            messages=prompt,
            temperature=0.7,
            max_tokens=8062,
            top_p=0.95,
            response_format={
                "type": "json_object"
            }
        )

        # Parse the response
        structured_data = response.to_dict().get("choices", [{}])[0].get("message", {}).get("content", "").strip()

        print(f"Output from LLM:")
        print(structured_data)
        return structured_data
    except Exception as e:
        print(f"Failed to parse response, Error: {str(e)}")
        return None

image_path_gpt = "https://smartcatalog.blob.core.windows.net/table-images/table-image.png"
gpt_output = process_image_gpt4o(image_path_gpt, prompt_text, few_shot_examples_gpt)

### The output i got from the above from GPT-4o
# {"Top":"CrystalArt CY01 / CY02","Base":"GFM69 / GFM73 - 06","Dimensions_CM":"240x120x74h","Dimensions_INCHES":"94¹/₂x47¹/₄x29¹/₈h","M3":"0,98","Colli":"3","EUR":"3.192"}


[{'role': 'user', 'content': [{'type': 'text', 'text': 'EXAMPLE 1:\nI want you to parse a furniture pricing table from the image attached and output the data in JSON format.\nThe table describes combinations of attributes, each represented by row and column headers. Your task is to produce the output below\n\nTASK INPUT: \nAn image of a complex table with pricing information based on several attributes, such as color codes, finishing, dimensions, etc.\nThere might be nested sub-categories and some columns might be combined\n\nTASK OUTPUT: \nA JSON array of objects, where each object represents the price for each combination of attributes in the table. The JSON format should be:\n[\n  {\n    attribute 1: attribute 1 value,\n    attribute 2: attribute 2 value,\n    attribute 3: attribute 3 value,\n    attribute X: attribute X value,\n    price: price in EUR from image\n  },\n  ... // More objects for other combinations\n]\n'}, {'type': 'image_url', 'image_url': {'url': 'https://smartcata

## Method 1 (old)
1-step processing using VLMs (not great results)
1. input table image
2. **produce all combinations** of attributes in json list of objects (can be 50-100+ objects)



### the prompts (old, unused)

In [ ]:
NUM_COMBINATIONS = 32

prompt="""
I want you to parse a furniture pricing table step-by-step from an image and output the data in JSON format.
The table describes combinations of attributes, each represented by row and column headers.

Here's the task:
INPUT:
An image of a complex table with pricing information based on several attributes: "Top" and "Base" color codes (separated by /), dimensions in cm and inches, volume (M3), Colli (presumably package quantity), and price (EUR).
There might be nested sub-categories and some columns might be combined (noted by the "/" symbol) if their pricing information is the same to save visual space but we need to unpack them to reveal all combinations

PROCESSING:
1. identify all combinations of attributes. If combined or nested, split each value based on the delimiter used ("/" in this case)
2. calculate and output the number of combinations total.
3. for each combination of attributes, identify the price and output all combinations based on the format below.

OUTPUT:
A JSON array of objects, where each object represents a unique combination of "Top," "Base," dimensions, and price. The JSON format should be:
[
  {
    "Top": "Top color from image",
    "Base": "Base color from image (with classification and/or number)",
    "Dimensions_CM": "Dimensions in cm from image",
    "Dimensions_INCHES": "Dimensions in inches from image",
    "M3": "M3 value from image",
    "Colli": "Colli value from image",
    "EUR": "EUR price from image"
  },
  ... // More objects for other combinations
]"""

new_prompt = f"""
I want you to parse a furniture pricing table step-by-step from an image and output the data in JSON format.
The table describes combinations of attributes, each represented by row and column headers.
There are {NUM_COMBINATIONS} total combinations, based on the calculation below

CALCULATION OF COMBINATIONS:
Here's the breakdown of combinations from the image:
1. Top:  CrystalArt CY01, CrystalArt CY02 (2 options)
2. Base: GFM69, GFM73, GFM11, GFM18 (4 options)
3. Dimensions (cm/inches): 240x120x74h, 240x120x74h sag, 300x120x74h, 300x120x74h sag (4 options -  Note: Though represented in both cm and inches, these are the same dimensions, just in different units, so they count as one attribute)
Calculation:
Total combinations = Top options * Base options * Dimension options
Total combinations = 2 * 4 * 4 = 32

TASK INPUT:
An image of a complex table with pricing information based on several attributes: "Top" and "Base" color codes (separated by /), dimensions in cm and inches, volume (M3), Colli (presumably package quantity), and price (EUR).
There might be nested sub-categories and some columns might be combined (noted by the "/" symbol) if their pricing information is the same to save visual space but we need to unpack them to reveal all combinations

TASK OUTPUT:
A JSON array of {NUM_COMBINATIONS} objects, where each object represents a unique combination of "Top," "Base," dimensions, and price. Each object has the following attributes:

ATTRIBUTES:
- "Top": "Top color from image",
- "Base": "Base color from image (with classification and/or number)",
- "Dimensions_CM": "Dimensions in cm from image",
- "Dimensions_INCHES": "Dimensions in inches from image",
- "M3": "M3 value from image",
- "Colli": "Colli value from image",
- "EUR": "EUR price from image"
"""

prompt_table_structure = """
You are analyzing a furniture pricing table in an image. You will pre-process this table
Your task is to extract all headers (aka attributes of the furniture) and basic structure of the table accurately. Here's what you need to do:

1. Identify all headers in the table, including those that are combined to save space. For eg some header values might be combined into 1 column separated by the forward slash "/"
2. Provide a JSON output for the headers, including all permutations or combinations of headers.

This output schema will be used to then extract actual pricing data for each combination of attributes.
"""

prompt_combination_count = """
I want you to parse a furniture pricing table step-by-step from an image and output the number of combinations of the data.
The table describes combinations of attributes, each represented by row and column headers.

Here's the task:
INPUT:
An image of a complex table with pricing information based on several attributes: "Top" and "Base" color codes (separated by /), dimensions in cm and inches, volume (M3), Colli (presumably package quantity), and price (EUR).
There might be nested sub-categories and some columns might be combined (noted by the "/" symbol) if their pricing information is the same to save visual space but we need to unpack them to reveal all combinations

PROCESSING:
1. identify all combinations of attributes. If combined or nested, split each value based on the delimiter used ("/" in this case)
2. calculate and output the number of combinations total.

OUTPUT:
The number of total combinations along with the formula used.
"""


### calling Gemini

In [ ]:
import os
import google.generativeai as genai
from google.colab import userdata


# Create the model
generation_config = {
  "response_mime_type": "application/json",
}

genai.configure(api_key=userdata.get('gemini_api_key'))
model = genai.GenerativeModel(
    "gemini-1.5-pro-002"
    # generation_config=generation_config
    # system_instruction=""
)

# UPLOAD TABLE IMAGE TO GEMINI
def upload_to_gemini(path, mime_type=None):
  # """Uploads the given file to Gemini.
  # See https://ai.google.dev/gemini-api/docs/prompting_with_media
  # """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def process_image_gemini(image_path, prompt, generation_config=None):
    # Upload the image to Gemini
    files = [
        upload_to_gemini(image_path, mime_type="image/png"),
    ]

    # Create the prompt object
    prompt_obj = {
        "parts": [
            files[0],
            prompt
        ]
    }

    # Generate content using the Gemini model
    if generation_config:
        response = model.generate_content(prompt_obj, generation_config=generation_config)
    else:
        response = model.generate_content(prompt_obj)

    print(response.text)
    return response.text


In [ ]:
image_data = process_image_gemini(local_path, new_prompt, generation_config)

Uploaded file 'table_image.png' as: https://generativelanguage.googleapis.com/v1beta/files/apte9sowvozl
[{"Top": "CrystalArt CY01", "Base": "GFM69", "Dimensions_CM": "240x120x74h", "Dimensions_INCHES": "94 1/2x47 1/4x29 1/8h", "M3": "0.98", "Colli": "3", "EUR": "3.192"}, {"Top": "CrystalArt CY01", "Base": "GFM73", "Dimensions_CM": "240x120x74h", "Dimensions_INCHES": "94 1/2x47 1/4x29 1/8h", "M3": "0.98", "Colli": "3", "EUR": "3.192"}, {"Top": "CrystalArt CY02", "Base": "GFM69", "Dimensions_CM": "240x120x74h", "Dimensions_INCHES": "94 1/2x47 1/4x29 1/8h", "M3": "0.98", "Colli": "3", "EUR": "3.211"}, {"Top": "CrystalArt CY02", "Base": "GFM73", "Dimensions_CM": "240x120x74h", "Dimensions_INCHES": "94 1/2x47 1/4x29 1/8h", "M3": "0.98", "Colli": "3", "EUR": "3.211"}, {"Top": "CrystalArt CY01", "Base": "GFM11", "Dimensions_CM": "240x120x74h", "Dimensions_INCHES": "94 1/2x47 1/4x29 1/8h", "M3": "0.98", "Colli": "3", "EUR": "3.211"}, {"Top": "CrystalArt CY01", "Base": "GFM18", "Dimensions_CM":

### Calling gpt4o


### file upload (unused)

In [ ]:
import requests
import os

# upload a image table to azure openai
def upload_file_to_azure_openai(file_path, purpose, api_key, endpoint):
    url = f"{endpoint}openai/files?api-version=2024-10-21"

    headers = {"api-key": api_key}

    # Validate file existence
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")

    # using openai azure client to directly upload
    with open(file_path, "rb") as image_file:
      file_response = client.files.create(
        file=image_file,
        purpose=purpose
    )
    # file_id = file_response.id
    return file_response

# call the function
file_path = "/content/table-image.png"  # Replace with your file path
purpose = "assistants"  # Purpose, such as fine-tuning or evaluation
response = upload_file_to_azure_openai(file_path, purpose, OAI_API_KEY, ENDPOINT)

if response:
    print("File uploaded successfully! Response:")
    print(response)
else:
    print("Failed to upload the file.")


File uploaded successfully! Response:
FileObject(id='assistant-f5K3gnNAPdGwAJDH6rMG4CyT', bytes=78771, created_at=1733101104, filename='table-image.png', object='file', purpose='assistants', status='processed', status_details=None)


### prompt definitions (new) & creation

### calling llm gpt-4o

In [ ]:
image_path = "/content/table-image.png"
image_data_gpt4o = process_image_gpt4o(image_path, messages)

Failed to parse response, Error: Error code: 400 - {'error': {'code': 'BadRequest', 'message': 'Invalid image URL. The URL must be a valid HTTP or HTTPS URL, or a data URL with base64 encoding.', 'param': None, 'type': None}}


In [ ]:
import json
import pandas as pd

gpt4o_json = json.loads(image_data_gpt4o).get("result")
print(gpt4o_json)
print(len(gpt4o_json))

pd.DataFrame.from_records(gpt4o_json)

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

## Misc, unused

### Misc below (unused stuff from Alex undatas)

In [ ]:
read_table_model = genai.GenerativeModel(
  model_name="gemini-1.5-pro",
  generation_config=generation_config,
  system_instruction="""Task Description: Read the table information in the image and extract the data requested by the user.
Image Description: The image contains a complex table.
Task Process Description: First, carefully examine the image to understand the table structure. Then, understand the user's request to extract the desired data. Finally, output the data in JSON format.
JSON Format:
[Extracted Data]""",
)

def get_image_top_data(image_path, part):
    # TODO Make these files available on the local file system
    # You may need to update the file paths
    files = [
      upload_to_gemini(image_path, mime_type="image/png"),
    ]
    chat_session = read_table_model.start_chat(
      history=[
        {
          "role": "user",
          "parts": [
            files[0],
            part,
          ],
        }
      ]
    )

    response = chat_session.send_message("INSERT_INPUT_HERE")

    print(response.text)
    return response.text

In [ ]:
get_image_top_data_part = "Get the 'Top' data from the table, split it using '/', and return the data in JSON format."
top_data = get_image_top_data(local_path, get_image_top_data_part)
top_data

Uploaded file 'table_image.png' as: https://generativelanguage.googleapis.com/v1beta/files/mxzre3wmwi4h
["CY01", "CY02"]


'["CY01", "CY02"]'

In [ ]:
top_list = json.loads(top_data)

table_json_data = []
for top in top_list:
   one_json_data = []
   for base in one_top_data:
      one_base = base.copy()
      one_base['Top'] = top
      one_json_data.append(one_base)
   table_json_data.extend(one_json_data)

print(table_json_data)
table_df = pd.DataFrame.from_records(table_json_data)
table_df

[{'Top': 'CY01', 'Base': 'GFM69-06', 'Dimensions_CM': '240x120x74h', 'Dimensions_INCHES': '94 1/2x47 1/4x29 1/8h', 'M3': '0.98', 'Colli': '3', 'EUR': '3.192'}, {'Top': 'CY01', 'Base': 'GFM73-06', 'Dimensions_CM': '240x120x74h', 'Dimensions_INCHES': '94 1/2x47 1/4x29 1/8h', 'M3': '0.98', 'Colli': '3', 'EUR': '3.192'}, {'Top': 'CY01', 'Base': 'GFM11-06', 'Dimensions_CM': '240x120x74h', 'Dimensions_INCHES': '94 1/2x47 1/4x29 1/8h', 'M3': '0.98', 'Colli': '3', 'EUR': '3.211'}, {'Top': 'CY01', 'Base': 'GFM18-06', 'Dimensions_CM': '240x120x74h', 'Dimensions_INCHES': '94 1/2x47 1/4x29 1/8h', 'M3': '0.98', 'Colli': '3', 'EUR': '3.211'}, {'Top': 'CY01', 'Base': 'GFM69-06', 'Dimensions_CM': '240x120x74h sag', 'Dimensions_INCHES': '94 1/2x47 1/4x29 1/8h shaped', 'M3': '0.98', 'Colli': '3', 'EUR': '3.370'}, {'Top': 'CY01', 'Base': 'GFM73-06', 'Dimensions_CM': '240x120x74h sag', 'Dimensions_INCHES': '94 1/2x47 1/4x29 1/8h shaped', 'M3': '0.98', 'Colli': '3', 'EUR': '3.370'}, {'Top': 'CY01', 'Base':

,Top,Base,Dimensions_CM,Dimensions_INCHES,M3,Colli,EUR
0,CY01,GFM69-06,240x120x74h,94 1/2x47 1/4x29 1/8h,0.98,3,3.192
1,CY01,GFM73-06,240x120x74h,94 1/2x47 1/4x29 1/8h,0.98,3,3.192
2,CY01,GFM11-06,240x120x74h,94 1/2x47 1/4x29 1/8h,0.98,3,3.211
3,CY01,GFM18-06,240x120x74h,94 1/2x47 1/4x29 1/8h,0.98,3,3.211
4,CY01,GFM69-06,240x120x74h sag,94 1/2x47 1/4x29 1/8h shaped,0.98,3,3.370
5,CY01,GFM73-06,240x120x74h sag,94 1/2x47 1/4x29 1/8h shaped,0.98,3,3.370
6,CY01,GFM11-06,240x120x74h sag,94 1/2x47 1/4x29 1/8h shaped,0.98,3,3.389
7,CY01,GFM18-06,240x120x74h sag,94 1/2x47 1/4x29 1/8h shaped,0.98,3,3.389
8,CY01,GFM69-06,300x120x74h,118 1/8x47 1/4x29 1/8h,1.12,4,3.785
9,CY01,GFM73-06,300x120x74h,118 1/8x47 1/4x29 1/8h,1.12,4,3.785


In [ ]:
table_df.to_excel('table_data.xlsx')